### **Aprendizaje profundo, optimización y atención**

Este cuaderno cubre **redes neuronales + optimización** y cierra con un **puente directo hacia Transformers**: *self-attention*, **conexiones residuales (skip connections)**, **capas de normalización** y **redes feed-forward** (FFN).



#### **Configuración**

Este cuaderno usa **NumPy + PyTorch** (CPU) y datasets sintéticos pequeños para que todo corra rápido.

In [ ]:
import math
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

# Reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


#### **1. Perceptrón (lineal) y MLP (no lineal)**

**Perceptrón**: modelo lineal con activación.
**MLP**: compone capas lineales + activaciones para modelar fronteras no lineales.

##### **1.1 Perceptrón "a mano" (NumPy)**

In [ ]:
def make_linearly_separable(n=200, margin=0.4):
    X = np.random.randn(n, 2)
    # Separación por una recta: x0 + x1 > 0
    y = (X[:,0] + X[:,1] > 0).astype(int)
    # Aumentamos el margen alejando puntos
    X = X + (2*y[:,None]-1)*margin*np.array([[1.0, 1.0]])
    return X, y

X, y = make_linearly_separable()

def perceptron_train(X, y, lr=0.1, epochs=25):
    w = np.zeros(X.shape[1])
    b = 0.0
    for _ in range(epochs):
        for xi, yi in zip(X, y):
            score = np.dot(w, xi) + b
            yhat = 1 if score >= 0 else 0
            err = yi - yhat
            w += lr * err * xi
            b += lr * err
    return w, b

w, b = perceptron_train(X, y, lr=0.05, epochs=20)

def perceptron_predict(X, w, b):
    return (X @ w + b >= 0).astype(int)

acc = (perceptron_predict(X, w, b) == y).mean()
acc


In [ ]:
# Visualización de frontera
xx = np.linspace(X[:,0].min()-1, X[:,0].max()+1, 200)
yy = np.linspace(X[:,1].min()-1, X[:,1].max()+1, 200)
XX, YY = np.meshgrid(xx, yy)
grid = np.c_[XX.ravel(), YY.ravel()]
Z = (grid @ w + b).reshape(XX.shape)

plt.figure()
plt.contourf(XX, YY, Z >= 0, alpha=0.3)
plt.scatter(X[:,0], X[:,1], c=y, s=15)
plt.title("Perceptrón (frontera lineal)")
plt.show()


##### **1.2 MLP (PyTorch) sobre un dataset no lineal (make_moons)**

In [ ]:
X, y = make_moons(n_samples=1200, noise=0.15, random_state=SEED)
X = X.astype(np.float32)
y = y.astype(np.int64)

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.4, random_state=SEED, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=SEED, stratify=y_tmp)

def to_tensor(a, dtype=None):
    t = torch.tensor(a)
    return t.to(dtype=dtype) if dtype is not None else t

X_train_t = to_tensor(X_train, torch.float32).to(device)
y_train_t = to_tensor(y_train, torch.long).to(device)
X_val_t   = to_tensor(X_val, torch.float32).to(device)
y_val_t   = to_tensor(y_val, torch.long).to(device)
X_test_t  = to_tensor(X_test, torch.float32).to(device)
y_test_t  = to_tensor(y_test, torch.long).to(device)

class MLP(nn.Module):
    def __init__(self, d_in=2, d_hidden=64, d_out=2, dropout_p=0.0, use_bn=False):
        super().__init__()
        self.fc1 = nn.Linear(d_in, d_hidden)
        self.bn1 = nn.BatchNorm1d(d_hidden) if use_bn else None
        self.drop = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(d_hidden, d_out)

    def forward(self, x):
        x = self.fc1(x)
        if self.bn1 is not None:
            x = self.bn1(x)
        x = F.relu(x)
        x = self.drop(x)
        return self.fc2(x)

def accuracy(logits, y):
    return (logits.argmax(dim=-1) == y).float().mean().item()

def train_classifier(model, optimizer, max_epochs=200, patience=20):
    loss_fn = nn.CrossEntropyLoss()
    best_val = float("inf")
    best_state = None
    bad = 0
    history = {"train_loss":[], "val_loss":[], "train_acc":[], "val_acc":[]}

    for _epoch in range(1, max_epochs+1):
        model.train()
        optimizer.zero_grad()
        logits = model(X_train_t)
        loss = loss_fn(logits, y_train_t)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            tr_logits = model(X_train_t)
            va_logits = model(X_val_t)
            tr_loss = loss_fn(tr_logits, y_train_t).item()
            va_loss = loss_fn(va_logits, y_val_t).item()

        history["train_loss"].append(tr_loss)
        history["val_loss"].append(va_loss)
        history["train_acc"].append(accuracy(tr_logits, y_train_t))
        history["val_acc"].append(accuracy(va_logits, y_val_t))

        # Early stopping
        if va_loss < best_val - 1e-4:
            best_val = va_loss
            best_state = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return history

model = MLP(dropout_p=0.0, use_bn=False).to(device)
opt = torch.optim.SGD(model.parameters(), lr=0.2)
hist = train_classifier(model, opt)

model.eval()
with torch.no_grad():
    test_logits = model(X_test_t)
test_acc = accuracy(test_logits, y_test_t)
test_acc


In [ ]:
plt.figure()
plt.plot(hist["train_loss"], label="train")
plt.plot(hist["val_loss"], label="val")
plt.title("Pérdida (baseline MLP)")
plt.legend()
plt.show()

plt.figure()
plt.plot(hist["train_acc"], label="train")
plt.plot(hist["val_acc"], label="val")
plt.title("Accuracy (baseline MLP)")
plt.legend()
plt.show()


#### **2. Activaciones y pérdidas: saturación y gradientes**

Sigmoid/tanh tienden a saturar -> gradientes pequeños. ReLU/GELU suelen ser más estables en profundidad.

##### **2.1 Norma de gradiente vs profundidad (intuición)**

In [ ]:
def grad_norm_through_depth(act="sigmoid", depth=30, width=128):
    x = torch.randn(1, width, device=device, requires_grad=True)
    layers = [nn.Linear(width, width).to(device) for _ in range(depth)]

    h = x
    for lin in layers:
        h = lin(h)
        if act == "sigmoid":
            h = torch.sigmoid(h)
        elif act == "tanh":
            h = torch.tanh(h)
        elif act == "relu":
            h = F.relu(h)
        elif act == "gelu":
            h = F.gelu(h)
        else:
            raise ValueError(act)

    loss = h.sum()
    loss.backward()
    return x.grad.norm().item()

acts = ["sigmoid","tanh","relu","gelu"]
depths = [10,20,30,40]
vals = {a: [grad_norm_through_depth(a, depth=d) for d in depths] for a in acts}

plt.figure()
for a in acts:
    plt.plot(depths, vals[a], marker="o", label=a)
plt.yscale("log")
plt.title("Norma de gradiente vs profundidad (escala log)")
plt.xlabel("profundidad (capas)")
plt.ylabel("||dL/dx||")
plt.legend()
plt.show()


## **3. Retropropagación y verificación por diferencias finitas**

La retropropagación calcula gradientes exactos (analíticos) usando regla de la cadena en reversa.  
Para verificar que la implementación es correcta, usamos una aproximación numérica:

$$
\frac{\partial L}{\partial \theta} \approx \frac{L(\theta+\epsilon)-L(\theta-\epsilon)}{2\epsilon}
$$

Compararemos el gradiente numérico con el gradiente de retropropagación (backprop) usando error relativo:

$$
\text{rel_err}=
\frac{\lVert g_{\text{num}}-g_{\text{backprop}} \rVert}{\lVert g_{\text{num}} \rVert+\lVert g_{\text{backprop}} \rVert + 10^{-12}}
$$


**Durante la verificación numérica de gradientes**, hacemos que el cálculo sea **determinista**: desactivamos dropout, fijamos batch normalization en modo evaluación (o congelamos sus estadísticas) y desactivamos el barajado de datos. Esto asegura que las evaluaciones de $L(\theta+\epsilon)$ y $L(\theta-\epsilon)$ sean comparables.



In [ ]:
def finite_diff_check(eps=1e-4):
    theta = torch.tensor([0.7], device=device, requires_grad=True)

    def L(t):
        return (torch.sin(t) + t**2).sum()

    loss = L(theta)
    loss.backward()
    grad_autograd = theta.grad.detach().cpu().item()

    theta_p = theta.detach().clone() + eps
    theta_m = theta.detach().clone() - eps
    grad_fd = ((L(theta_p) - L(theta_m)) / (2*eps)).detach().cpu().item()
    return grad_autograd, grad_fd

finite_diff_check()


#### **4. Regularización y generalización (L2, dropout, batch norm, early stopping)**


A continuación se comparan en **make_moons** (mismo modelo, cambios mínimos).

In [ ]:
def run_experiment(tag, dropout_p=0.0, use_bn=False, optimizer_name="sgd", lr=0.2, weight_decay=0.0):
    model = MLP(dropout_p=dropout_p, use_bn=use_bn).to(device)

    if optimizer_name == "sgd":
        opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "adam":
        opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "adamw":
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(optimizer_name)

    hist = train_classifier(model, opt, max_epochs=400, patience=30)

    model.eval()
    with torch.no_grad():
        test_acc = accuracy(model(X_test_t), y_test_t)

    return tag, hist, test_acc

experiments = [
    ("baseline", 0.0, False, "sgd", 0.2, 0.0),
    ("L2 (wd=1e-3)", 0.0, False, "sgd", 0.2, 1e-3),
    ("dropout(0.2)", 0.2, False, "sgd", 0.2, 0.0),
    ("batchnorm", 0.0, True, "sgd", 0.2, 0.0),
]

results = []
for tag, dp, bn, optn, lr, wd in experiments:
    results.append(run_experiment(tag, dp, bn, optn, lr, wd))

[(tag, acc) for tag,_,acc in results]


In [ ]:
plt.figure()
for tag, hist, _ in results:
    plt.plot(hist["val_loss"], label=tag)
plt.title("Validación: pérdida vs época (regularización)")
plt.legend()
plt.show()


#### **5. Optimizadores modernos y programación de tasa de aprendizaje**

Mostremos el **SGD con momentum, Adam/AdamW, y schedules**.

##### **5.1 Comparación: SGD+momentum vs Adam vs AdamW**

In [ ]:
opt_exps = [
    ("SGD+mom", 0.0, False, "sgd", 0.2, 1e-4),
    ("Adam",    0.0, False, "adam", 3e-3, 1e-4),
    ("AdamW",   0.0, False, "adamw", 3e-3, 1e-2),
]

opt_results = []
for tag, dp, bn, optn, lr, wd in opt_exps:
    opt_results.append(run_experiment(tag, dp, bn, optn, lr, wd))

[(tag, acc) for tag,_,acc in opt_results]


In [ ]:
plt.figure()
for tag, hist, _ in opt_results:
    plt.plot(hist["train_loss"], label=tag)
plt.title("Entrenamiento: pérdida vs época (optimizadores)")
plt.legend()
plt.show()


##### **5.2 Scheduler de ejemplo (cosine)**

Un *learning-rate scheduler* ajusta la tasa de aprendizaje (LR) **durante el entrenamiento**. La razón es práctica: con una LR fija, a menudo:

- al inicio quieres una LR relativamente alta para **avanzar rápido**,
- cerca del final quieres una LR más baja para **refinar** (evitar "rebotar" alrededor del mínimo).

**CosineAnnealingLR** implementa un descenso suave de la LR siguiendo una curva coseno. En su forma estándar, la LR disminuye desde `lr_inicial` hacia `eta_min` (por defecto 0) en `T_max` pasos:

$$
\text{lr}(t) = \eta_{\min} + \frac{1}{2}(\eta_{\max}-\eta_{\min})\left(1+\cos\left(\pi \frac{t}{T_{\max}}\right)\right)
$$

- $\eta_{\max}$: LR inicial (la que le pasas al optimizador, por ejemplo `lr=3e-3`).
- $\eta_{\min}$: LR mínima (si no se especifica, suele ser 0).
- $t$: paso actual del scheduler (aquí lo hacemos por *step*).
- $T_{\max}$: número total de pasos para "completar" la caída coseno.






In [ ]:
model = MLP(dropout_p=0.1, use_bn=False).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=3e-3, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=100)

loss_fn = nn.CrossEntropyLoss()
lrs, losses = [], []

model.train()
for step in range(100):
    opt.zero_grad()
    logits = model(X_train_t)
    loss = loss_fn(logits, y_train_t)
    loss.backward()
    opt.step()
    scheduler.step()
    lrs.append(opt.param_groups[0]["lr"])
    losses.append(loss.item())

plt.figure()
plt.plot(losses)
plt.title("Loss (con cosine schedule)")
plt.show()

plt.figure()
plt.plot(lrs)
plt.title("Learning rate (cosine schedule)")
plt.show()


#### **6. Limitaciones de RNN/LSTM en secuencias largas y motivación hacia atención**

##### **6.1 Experimento: norma de gradiente vs longitud en una RNN simple**

In [ ]:
class TinyRNN(nn.Module):
    def __init__(self, d_in=8, d_h=32, d_out=8):
        super().__init__()
        self.Wx = nn.Linear(d_in, d_h, bias=False)
        self.Wh = nn.Linear(d_h, d_h, bias=False)
        self.Wo = nn.Linear(d_h, d_out)

    def forward(self, x):
        # x: [T, B, d_in]
        h = torch.zeros(x.size(1), self.Wh.in_features, device=x.device)
        for t in range(x.size(0)):
            h = torch.tanh(self.Wx(x[t]) + self.Wh(h))
        return self.Wo(h)

def grad_norm_rnn(T=50):
    rnn = TinyRNN().to(device)
    x = torch.randn(T, 16, 8, device=device)
    y = torch.randint(0, 8, (16,), device=device)

    logits = rnn(x)
    loss = nn.CrossEntropyLoss()(logits, y)
    loss.backward()
    return rnn.Wh.weight.grad.norm().item()

Ts = [10, 20, 40, 80, 120, 160]
gn = [grad_norm_rnn(T) for T in Ts]

plt.figure()
plt.plot(Ts, gn, marker="o")
plt.yscale("log")
plt.title("RNN: norma de gradiente vs longitud T (escala log)")
plt.xlabel("T")
plt.ylabel("||grad||")
plt.show()

gn


#### **RNN/LSTM/Seq2Seq (traducción): puente hacia atención**

En esta sección integrada trabajamos un modelo **secuencia-a-secuencia** recurrente para traducción y observamos, en práctica, por qué las RNN/LSTM:

- se vuelven frágiles con **dependencias largas** (propagación de gradientes y "cuello de botella" del estado oculto),
- requieren trucos como **teacher forcing** y estrategias de decodificación,
- motivan un mecanismo explícito de **atención** para acceder a todo el contexto sin comprimirlo en un único vector.


### **Modelos RNN de secuencia a secuencia: Tarea de traducción**


#### **Configuración**


In [ ]:
# Todas las librería necesarias para este cuaderno  están listadas a continuación.  

#!mamba install -qy numpy==1.21.4 seaborn==0.9.0  
#**Nota**: Si tu entorno no admite `!mamba install`, utiliza `!pip install`.


Las siguientes librerías necesarias **no** están preinstaladas en el entorno del curso. **Necesitarás ejecutar la siguiente celda** para instalarlas:  


In [ ]:
#!pip install torch==2.0.0
#!pip install spacy==3.7.2
#!pip install nltk==3.8.1
#!pip install -U matplotlib

!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


#### **Importación de librerías necesarias**  
_Se recomienda importar todas las librerías necesarias en un solo lugar (aquí):_



In [ ]:
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from nltk.translate.bleu_score import sentence_bleu
import torch
import torch.nn as nn
import torch.optim as optim


import numpy as np
import random
import math
import time
from tqdm import tqdm
import matplotlib.pyplot as plt


# Usamos esta sección para suprimir warnings generados por el codigo:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


#### **Introducción a las RNN**

Las RNN son una clase de redes neuronales diseñadas para procesar datos secuenciales.   Mantienen una memoria interna ($h_t$) para capturar información de pasos anteriores y utilizarla en predicciones actuales.  

Las redes neuronales recurrentes (RNN) operan sobre secuencias y utilizan estados anteriores para influir en el estado actual. Aquí está la formulación general de una RNN simple:

Dado:

- $ \mathbf{x}_t $: vector de entrada en el instante de tiempo $t$

- $ \mathbf{h}_{t-1} $: vector del estado oculto del paso de tiempo anterior

- $ \mathbf{W}_x $ y $ \mathbf{W}_h $: matrices de pesos para la entrada y el estado oculto, respectivamente

- $ \mathbf{b} $: vector de sesgo

- $\sigma$: función de activación (a menudo una sigmoide o tanh)

Las ecuaciones de actualización para el estado oculto $ \mathbf{h}_t $ y la salida $ \mathbf{y}_t $ son las siguientes:

$$
\begin{align*}
\mathbf{h}_t &= \sigma(\mathbf{W}_x \cdot \mathbf{x}_t + \mathbf{W}_h \cdot \mathbf{h}_{t-1} + \mathbf{b})
\end{align*}
$$

Puede verse que la función de estado oculto depende del estado oculto anterior así como de la entrada en el tiempo $t$, por lo que tiene una memoria colectiva de los pasos anteriores.

Para la salida (si estamos haciendo una predicción en cada paso de tiempo):

$$
\begin{align*}
\mathbf{y}_t &= \text{softmax}(\mathbf{W}_o \cdot \mathbf{h}_t + \mathbf{b}_o)
\end{align*}
$$

Donde:

$ \mathbf{W}_o $: matriz de pesos para la salida Y $ \mathbf{b}_o $: vector de sesgo para la salida

Dependiendo de la tarea específica, una celda RNN puede producir una salida desde $h_t$ o simplemente transferirla a la siguiente celda, actuando como memoria interna.  
Aunque la capacidad de la arquitectura para retener memoria pueda parecer esquiva a primera vista, aclaremos esto implementando una RNN simple para manejar el siguiente mecanismo de datos:

![a title](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Screenshot%202023-10-19%20at%2011.29.23%E2%80%AFAM.png)

El diagrama muestra una máquina de estados o modelo de transición con tres estados distintos, representados por los círculos púrpuras prominentes. Cada estado está claramente etiquetado con un valor para $ h $: $ h = -1 $, $ h = 0 $, y $ h = 1 $.

1. **Estado $ h = -1 $**:
   - Se mantiene en sí mismo cuando $ x = 1 $ (ilustrado por el bucle amarillo).
   - Pasa al estado $ h = 0$ al recibir $ x = -1$ (destacado por la flecha roja).

2. **Estado $ h = 0 $**:
   - Se mueve al estado $h = -1 $ cuando $ x = 1$ (ilustrado por la flecha roja).
   - Avanza al estado $ h = 1 $ con $ x = -1$ (marcado por la flecha roja).

3. **Estado $h = 1 $**:
   - Mantiene su posición cuando $ x = -1 $ (indicado por el bucle amarillo).
   - Transiciona al estado $ h = 0 $ al recibir $ x = 1 $ (señalado por la flecha roja).

Para resumir, el diagrama representa eficazmente las transiciones entre tres estados basadas en la entrada $ x $.  
Dependiendo del estado actual y de la entrada $ x $, la máquina de estados transiciona a un estado diferente o permanece estacionaria.

Podemos  representar la máquina de estados mencionada anteriormente, para ello usamos $tanh$ ya que el valor de $h$ debe estar entre `[-1, 1]`. 

>Tenemos en cuenta que hemos excluido la salida para simplificar.

$$\begin{align*}
W_{xh} & = -10.0 \\\\\
W_{hh} & = 10.0 \\
b_h & = 0.0 \\
x_t & = 1 \\
h_{\text{prev}} & = 0.0 \\
h_t & = \tanh(x_t \cdot W_{xh} + h_{\text{prev}} \cdot W_{hh} + b_h)
\end{align*}$$


In [ ]:
W_xh=torch.tensor(-10.0)
W_hh=torch.tensor(10.0)
b_h=torch.tensor(0.0)
x_t=1
h_prev=torch.tensor(-1)


Consideramos la siguiente secuencia $x_t$ para  $t=0,1,..,7$,


In [ ]:
X=[1,1,-1,-1,1,1]


Asumiendo que comenzamos desde el estado inicial $h = 0$ con el vector de entrada $x$ anterior, el vector de estados $h$ debería verse así:


In [ ]:
H=[-1,-1,0,1,0,-1]


In [ ]:
# Inicializa una lista vacía para almacenar los valores del estado predicho
H_hat = []

# Variable auxiliar para llevar el control del tiempo t
t = 1

# Itera sobre cada punto de datos en la secuencia de entrada X
for x in X:
    # Imprime el instante de tiempo actual
    print("t=", t)
    
    # Asigna el valor del punto de datos actual a x_t
    x_t = x

    # Imprime el valor del estado anterior (h en el tiempo t-1)
    print("h_t-1", h_prev.item())

    # Calcula el estado actual (h en el tiempo t) utilizando la fórmula de la RNN con activación tanh
    h_t = torch.tanh(x_t * W_xh + h_prev * W_hh + b_h)

    # Actualiza h_prev con el estado actual para usarlo en la próxima iteración
    h_prev = h_t

    # Imprime el valor de entrada actual (x en el tiempo t)
    print("x_t", x_t)

    # Imprime el valor del estado actual (h en el tiempo t)
    print("h_t", h_t.item())
    print("\n")

    # Convierte h_t a entero y lo agrega a la lista H_hat
    H_hat.append(int(h_t.item()))

    # Incrementa el tiempo
    t += 1


Podemos evaluar la precisión del estado predicho ```H_hat``` comparándolo con el estado real ```H```. En las RNN, el estado $ h_t $ se utiliza para predecir una secuencia de salida $ y_t $ basada en la secuencia de entrada dada $ x_t $.


In [ ]:
H_hat


In [ ]:
H


Aunque hemos predefinido los valores de $W_{xh}$, $W_{hh}$ y $b_h$, en la práctica estos valores deben ser determinados mediante entrenamiento con datos. 

En la práctica, se utilizan con frecuencia modificaciones y mejoras, como LSTM (Long Short-Term Memory) y GRU (Gated Recurrent Units) para abordar problemas como el desvanecimiento del gradiente en las RNN básicas.


Una celda LSTM tiene tres componentes principales: una puerta de entrada, una puerta de olvido y una puerta de salida.  

- La **puerta de entrada** controla cuánta información nueva debe almacenarse en la memoria de la celda. Observa la entrada actual y el estado oculto anterior y decide qué partes de la nueva entrada deben recordarse.  

- La **puerta de olvido** determina qué información debe descartarse u olvidarse de la memoria de la celda. Considera la entrada actual y el estado oculto anterior y decide qué partes de la memoria anterior ya no son relevantes.  

- La **puerta de salida** determina qué información debe salir de la celda. Observa la entrada actual y el estado oculto anterior y decide qué partes de la memoria de la celda deben incluirse en la salida.

La idea clave detrás de las celdas LSTM es que tienen un estado de memoria separado que puede retener o olvidar información selectivamente con el tiempo. Esto les ayuda a manejar dependencias de largo alcance y recordar información importante de pasos anteriores en una secuencia.

#### **Arquitectura secuencia a secuencia**

Los modelos Seq2seq tienen una estructura de codificador-decodificador. El codificador codifica la secuencia de entrada en una representación de dimensión fija, a menudo llamada vector de contexto ($h_t$). El decodificador genera la secuencia de salida basada en el vector de contexto codificado.



<video width="640" height="480"
src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Translation_RNN.mp4"
controls>
</video>


#### **Implementación del codificador en PyTorch**


**Nota**: Al usar un LSTM, disponemos de un estado de celda adicional. Sin embargo, si utilizamos un GRU, solo tendríamos el estado oculto.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hid_dim, n_layers, dropout_prob):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # Capa de embedding: convierte índices de palabras en vectores de embedding
        self.embedding = nn.Embedding(vocab_len, emb_dim)

        # Capa LSTM: recibe embeddings y genera salidas y estados
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout_prob)

        # Capa de dropout para regularización
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, input_batch):
        # input_batch = [longitud de la secuencia fuente, tamaño del lote]

        # Aplica embedding seguido de dropout
        embed = self.dropout(self.embedding(input_batch))
        embed = embed.to(device)

        # outputs = [longitud de la secuencia fuente, tamaño del lote, dimensión oculta * número de direcciones]
        # hidden = [número de capas * número de direcciones, tamaño del lote, dimensión oculta]
        # cell = [número de capas * número de direcciones, tamaño del lote, dimensión oculta]
        outputs, (hidden, cell) = self.lstm(embed)

        return hidden, cell


Ahora estamos listo para crear una instancia del codificador y ver cómo funciona.


In [ ]:
vocab_len = 8
emb_dim = 10
hid_dim=8
n_layers=1
dropout_prob=0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder_t = Encoder(vocab_len, emb_dim, hid_dim, n_layers, dropout_prob).to(device)


Veamos un ejemplo simple donde el método `forward` del codificador transforma la oración `src` en los estados `hidden` y `cell`.  

El tensor `tensor([[0],[3],[4],[2],[1]])` es equivalente a `src` = 0,3,4,2,1, en el cual cada número representa un token en el vocabulario de `src`.  

Por ejemplo: 0:`<bos>`, 3:"Das", 4:"ist", 2:"schön", 1:`<eos>`.  

Tenemos un tamaño de lote (*batch size*) de 1.

In [ ]:
src_batch = torch.tensor([[0,3,4,2,1]])
# Necesitas transponer el tensor de entrada ya que el LSTM del codificador está en modo "secuencia primero" por defecto
src_batch = src_batch.t().to(device)
print("Forma del tensor de entrada (src):", src_batch.shape)
hidden_t , cell_t = encoder_t(src_batch)
print("Tensor oculto (hidden) del codificador:", hidden_t, "\nTensor de celda (cell) del codificador:", cell_t)


El codificador toma toda la secuencia de origen como entrada, la cual consiste en una secuencia de palabras o tokens. El LSTM del codificador procesa toda la secuencia de entrada y actualiza sus estados ocultos en cada paso de tiempo. Los estados ocultos de la red LSTM actúan como una forma de memoria y capturan la información contextual de la secuencia de entrada. Después de procesar toda la secuencia de entrada, el estado oculto final del LSTM del codificador captura la representación resumida del contexto de la secuencia de entrada. Este estado oculto final a veces se conoce como  **vector de contexto**.


#### **Implementación del decodificador en PyTorch**

Para comprender mejor el mecanismo interno de la parte del decodificador, echemos un vistazo más detallado:


<video width="640" height="480"
       src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/decoder_RNN.mp4"
       controls>
</video>


In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # Capa de embedding para convertir índices en vectores densos
        self.embedding = nn.Embedding(output_dim, emb_dim)

        # Capa LSTM que procesa la secuencia embebida
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        # Capa totalmente conectada que proyecta a la dimensión del vocabulario
        self.fc_out = nn.Linear(hid_dim, output_dim)

        # Capa de softmax logarítmica para obtener probabilidades
        self.softmax = nn.LogSoftmax(dim=1)

        # Capa de dropout para regularización
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        # input = [tamaño del lote]

        # hidden = [n capas * n direcciones, tamaño del lote, dim oculta]
        # cell = [n capas * n direcciones, tamaño del lote, dim oculta]

        # En el decodificador, el número de direcciones siempre será 1, por lo tanto:
        # hidden = [n capas, tamaño del lote, dim oculta]
        # cell = [n capas, tamaño del lote, dim oculta]

        # Añade una dimensión de secuencia (tamaño 1) al input
        input = input.unsqueeze(0)
        # input = [1, tamaño del lote]

        # Aplica embedding y dropout
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, tamaño del lote, dim embedding]

        # Pasa por la LSTM
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output = [long secuencia, tamaño del lote, dim oculta * n direcciones]
        # hidden = [n capas * n direcciones, tamaño del lote, dim oculta]
        # cell = [n capas * n direcciones, tamaño del lote, dim oculta]

        # Como la longitud de la secuencia y el número de direcciones son 1:
        # output = [1, tamaño del lote, dim oculta]
        # hidden = [n capas, tamaño del lote, dim oculta]
        # cell = [n capas, tamaño del lote, dim oculta]

        # Aplica capa lineal y softmax logarítmico para obtener la predicción
        prediction_logit = self.fc_out(output.squeeze(0))
        prediction = self.softmax(prediction_logit)
        # prediction = [tamaño del lote, dimensión de salida]

        return prediction, hidden, cell


Podemos crear una instancia del decodificador. La dimensión de salida se establece como la longitud del vocabulario objetivo.

In [ ]:
output_dim = 6
emb_dim=10
hid_dim = 8
n_layers=1
dropout=0.5
decoder_t = Decoder(output_dim, emb_dim, hid_dim, n_layers, dropout).to(device)


Ahora que tenemos instancias tanto del codificador como del decodificador, estamos listo para conectarlos (la caja roja en el diagrama a continuación). Primero, veamos cómo podemos pasar los estados *Hidden* y *Cell* (la celda rosada dentro de la caja roja) del codificador (el contenedor de cajas verdes) al decodificador (el contenedor de cajas naranjas). 


![connection](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/ED_connection.JPG)


In [ ]:
input_t = torch.tensor([0]).to(device) #<bos>
input_t.shape
prediction, hidden, cell = decoder_t(input_t, hidden_t , cell_t)
print("Prediccion:", prediction, '\nOculto:',hidden,'\nCelda:', cell)


#### **Conexión codificador-decodificador** 


Aprendimos cómo crear los módulos del codificador y del decodificador y cómo pasarles entradas. Ahora necesitamos crear la conexión para que el modelo pueda procesar pares (`src`, `trg`) y generar la traducción. Supongamos que `trg` es el tensor `([[0],[2],[3],[5],[1]])`, lo cual es equivalente a la secuencia 0,2,3,5,1, en la que cada número representa un token en el vocabulario objetivo.

Por ejemplo: 0:`<bos>`, 2:"this", 3:"is", 5:"beautiful", 1:`<eos>`.


In [ ]:
# trg = [longitud de trg, tamaño del lote]
# teacher_forcing_ratio es la probabilidad de usar teacher forcing
# por ejemplo, si teacher_forcing_ratio es 0.75, usas entradas reales (ground-truth) el 75% del tiempo
teacher_forcing_ratio = 0.5
trg = torch.tensor([[0],[2],[3],[5],[1]]).to(device)

# Obtiene el tamaño del lote desde la dimensión 1 de trg
batch_size = trg.shape[1]

# Obtiene la longitud de la secuencia trg
trg_len = trg.shape[0]

# Obtiene el tamaño del vocabulario de salida del decodificador
trg_vocab_size = decoder_t.output_dim

# Tensor para almacenar las salidas del decodificador
outputs_t = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

# Envía los tensores hidden y cell al dispositivo (CPU o GPU)
hidden_t = hidden_t.to(device)
cell_t = cell_t.to(device)

# la primera entrada para el decodificador es el token <bos>
input = trg[0, :]

# Bucle sobre la longitud de la secuencia de salida
for t in range(1, trg_len):

    # Se recorre la longitud de trg y se generan tokens
    # El decodificador recibe el token anterior generado, el estado de celda y el estado oculto
    # El decodificador produce la predicción (distribución de probabilidad del siguiente token) y actualiza hidden y cell
    output_t, hidden_t, cell_t = decoder_t(input, hidden_t, cell_t)

    # Guarda la predicción en el tensor que almacena las salidas del decodificador
    outputs_t[t] = output_t

    # Decide si se usará teacher forcing
    teacher_force = random.random() < teacher_forcing_ratio

    # Obtiene el token con mayor probabilidad entre las predicciones
    top1 = output_t.argmax(1)

    # Si se usa teacher forcing, se utiliza el token real como siguiente entrada
    # Si no, se utiliza el token predicho
    input = trg[t] if teacher_force else top1

# Muestra las salidas del decodificador y su forma
print(outputs_t, outputs_t.shape)


El tamaño del tensor de salida es `(trg_len, batch_size, trg_vocab_size)`. Esto se debe a que, para cada token de `trg` (longitud de `trg`), el modelo genera una distribución de probabilidad sobre todos los posibles tokens (longitud del vocabulario de `trg`). Por lo tanto, para generar los tokens predichos o la traducción de la oración `src`, necesitas obtener la probabilidad máxima para cada token:

In [ ]:
# Ten en cuenta que necesitamos obtener el `argmax` de la segunda dimensión, ya que **outputs** es un 
# arreglo de tensores de **salida**.
pred_tokens = outputs_t.argmax(2)
print(pred_tokens)


No es sorprendente que la traducción no sea correcta (`trg = tensor([[0],[2],[3],[5],[1]])`), ya que el modelo aún no ha pasado por ningún entrenamiento. Vamos a reunir todo el código para conectar el codificador y el decodificador en una clase `seq2seq` para una mejor usabilidad.


#### **Implementación del modelo secuencia a secuencia en PyTorch**

Vamos a conectar los componentes del codificador y el decodificador para crear el modelo `seq2seq`.


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, trg_vocab):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.trg_vocab = trg_vocab

        # Asegúrate de que las dimensiones ocultas del codificador y del decodificador sean iguales
        assert encoder.hid_dim == decoder.hid_dim, \
            "¡Las dimensiones ocultas del codificador y del decodificador deben ser iguales!"
        # Asegúrate de que ambos tengan el mismo número de capas
        assert encoder.n_layers == decoder.n_layers, \
            "¡El codificador y el decodificador deben tener el mismo número de capas!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        # src = [longitud de src, tamaño del lote]
        # trg = [longitud de trg, tamaño del lote]
        # teacher_forcing_ratio es la probabilidad de usar teacher forcing
        # por ejemplo, si teacher_forcing_ratio es 0.75, usas la entrada real el 75% del tiempo

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # Tensor para almacenar las salidas del decodificador
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # El último estado oculto del codificador se usa como estado inicial del decodificador
        hidden, cell = self.encoder(src)
        hidden = hidden.to(device)
        cell = cell.to(device)

        # la primera entrada del decodificador son los tokens <bos>
        input = trg[0, :]

        for t in range(1, trg_len):

            # Inserta el embedding del token de entrada, estado oculto y de celda anteriores
            # Recibe el tensor de salida (predicciones) y los nuevos estados oculto y de celda
            output, hidden, cell = self.decoder(input, hidden, cell)

            # Guarda las predicciones en un tensor que almacena las predicciones para cada token
            outputs[t] = output

            # Decide si se va a usar teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio

            # Obtiene el token con mayor probabilidad de las predicciones
            top1 = output.argmax(1)

            # Si se usa teacher forcing, usa el siguiente token real como entrada
            # Si no, usa el token predicho
            input = trg[t] if teacher_force else top1

        return outputs


#### **Entrenamiento del modelo en PyTorch**



In [ ]:
def train(modelo, iterator, optimizer, criterion, clip):

    # Establece el modelo en modo de entrenamiento
    modelo.train()

    # Inicializa la pérdida acumulada de la época
    epoch_loss = 0

    # Envuelve el iterador con tqdm para mostrar el progreso durante el entrenamiento
    train_iterator = tqdm(iterator, desc="Entrenando", leave=False)

    # Itera sobre los lotes del conjunto de entrenamiento
    for i, (src, trg) in enumerate(iterator):

        # Envía los datos al dispositivo (CPU o GPU)
        src = src.to(device)
        trg = trg.to(device)

        # Reinicia los gradientes del optimizador
        optimizer.zero_grad()

        # Ejecuta el modelo sobre las secuencias de entrada y objetivo
        output = modelo(src, trg)

        # trg = [longitud de trg, tamaño del lote]
        # output = [longitud de trg, tamaño del lote, dimensión de salida]

        output_dim = output.shape[-1]

        # Excluye el primer token (<bos>) de la salida y reestructura el tensor
        output = output[1:].view(-1, output_dim)

        # Excluye también el primer token (<bos>) de trg y lo convierte en un tensor contiguo plano
        trg = trg[1:].contiguous().view(-1)

        # trg = [(longitud de trg - 1) * tamaño del lote]
        # output = [(longitud de trg - 1) * tamaño del lote, dimensión de salida]

        # Calcula la pérdida entre la salida del modelo y el valor real
        loss = criterion(output, trg)

        # Propagación hacia atrás: calcula los gradientes
        loss.backward()

        # Aplica recorte de gradientes para evitar explosiones de gradientes
        torch.nn.utils.clip_grad_norm_(modelo.parameters(), clip)

        # Actualiza los parámetros del modelo
        optimizer.step()

        # Actualiza la barra de progreso de tqdm con la pérdida actual
        train_iterator.set_postfix(loss=loss.item())

        # Acumula la pérdida del lote actual
        epoch_loss += loss.item()

    # Devuelve la pérdida promedio por lote en toda la época
    return epoch_loss / len(list(iterator))


#### **Evaluación del modelo en PyTorch**

También necesitamos definir una función para evaluar el modelo. 

In [ ]:
def evaluate(modelo, iterator, criterion):

    # Establece el modelo en modo evaluación
    modelo.eval()

    # Inicializa la pérdida acumulada de la época
    epoch_loss = 0

    # Envuelve el iterador con tqdm para mostrar el progreso durante la evaluación
    valid_iterator = tqdm(iterator, desc="Evaluando", leave=False)

    # Desactiva el cálculo de gradientes para ahorrar memoria y acelerar la evaluación
    with torch.no_grad():

        # Itera sobre los lotes del conjunto de evaluación
        for i, (src, trg) in enumerate(iterator):

            # Envía los datos al dispositivo (CPU o GPU)
            src = src.to(device)
            trg = trg.to(device)

            # Ejecuta el modelo con teacher forcing desactivado
            output = modelo(src, trg, 0)  # desactiva el teacher forcing

            # trg = [longitud de trg, tamaño del lote]
            # output = [longitud de trg, tamaño del lote, dimensión de salida]

            output_dim = output.shape[-1]

            # Excluye el primer token (<bos>) y reestructura la salida
            output = output[1:].view(-1, output_dim)

            # Excluye también el primer token (<bos>) de trg y lo convierte en tensor plano
            trg = trg[1:].contiguous().view(-1)

            # trg = [(longitud de trg - 1) * tamaño del lote]
            # output = [(longitud de trg - 1) * tamaño del lote, dimensión de salida]

            # Calcula la pérdida entre la salida del modelo y el valor real
            loss = criterion(output, trg)

            # Actualiza la barra de progreso de tqdm con la pérdida actual
            valid_iterator.set_postfix(loss=loss.item())

            # Acumula la pérdida del lote actual
            epoch_loss += loss.item()

    # Devuelve la pérdida promedio por lote durante la evaluación
    return epoch_loss / len(list(iterator))


### **Tarea de traducción**

#### **1. Preprocesamiento con spaCy + dataset mínimo (EN->DE)**

In [ ]:
import random
import numpy as np
import torch
import spacy

# Tokenizadores spaCy (EN y DE)
spacy_en = spacy.load("en_core_web_sm")
spacy_de = spacy.load("de_core_news_sm")

def tokenize_en(texto: str):
    """Tokeniza en inglés (minúsculas)."""
    return [tok.text.lower() for tok in spacy_en.tokenizer(texto)]

def tokenize_de(texto: str):
    """Tokeniza en alemán (minúsculas)."""
    return [tok.text.lower() for tok in spacy_de.tokenizer(texto)]

#  Dataset paralelo pequeño
pairs = [
    ("i am a student", "ich bin ein student"),
    ("i am a teacher", "ich bin ein lehrer"),
    ("he is a student", "er ist ein student"),
    ("she is a teacher", "sie ist eine lehrerin"),
    ("i like apples", "ich mag äpfel"),
    ("i like books", "ich mag bücher"),
    ("we like apples", "wir mögen äpfel"),
    ("they like books", "sie mögen bücher"),
    ("good morning", "guten morgen"),
    ("good night", "gute nacht"),
    ("thank you", "danke"),
    ("you are welcome", "bitte"),
    ("how are you", "wie geht es dir"),
    ("i am fine", "mir geht es gut"),
]

# Split determinístico (evita que justo tus ejemplos queden en valid y el modelo colapse) ---
train_pairs = pairs[:12]
valid_pairs = pairs[12:]

print("Tamaño train:", len(train_pairs))
print("Tamaño valid:", len(valid_pairs))
print("\nEjemplos TRAIN:", train_pairs[:3])
print("Ejemplos VALID:", valid_pairs[:3])


#### **2. Vocabulario propio**

In [ ]:
# Tokens especiales
PAD = "<pad>"
BOS = "<bos>"
EOS = "<eos>"
UNK = "<unk>"

class Vocab:
    """Vocabulario mínimo: stoi/itos + encode/decode."""
    def __init__(self, listas_tokens, min_freq=1):
        freq = {}
        for toks in listas_tokens:
            for t in toks:
                freq[t] = freq.get(t, 0) + 1

        # Orden fijo: especiales primero
        self.itos = [PAD, BOS, EOS, UNK]

        # Resto por frecuencia y orden alfabético
        for t, c in sorted(freq.items(), key=lambda x: (-x[1], x[0])):
            if c >= min_freq and t not in self.itos:
                self.itos.append(t)

        self.stoi = {t: i for i, t in enumerate(self.itos)}

        self.pad_idx = self.stoi[PAD]
        self.bos_idx = self.stoi[BOS]
        self.eos_idx = self.stoi[EOS]
        self.unk_idx = self.stoi[UNK]

    def __len__(self):
        return len(self.itos)

    def encode(self, tokens):
        return [self.stoi.get(t, self.unk_idx) for t in tokens]

    def decode(self, ids):
        return [self.itos[i] if i < len(self.itos) else UNK for i in ids]

#  Construir vocab SOLO con train (práctica correcta)
src_token_lists = [tokenize_en(s) for s, _ in train_pairs]
trg_token_lists = [tokenize_de(t) for _, t in train_pairs]

SRC_VOCAB = Vocab(src_token_lists, min_freq=1)
TRG_VOCAB = Vocab(trg_token_lists, min_freq=1)

print("SRC vocab size:", len(SRC_VOCAB))
print("TRG vocab size:", len(TRG_VOCAB))
print("TRG PAD idx:", TRG_VOCAB.pad_idx)


#### **3.Dataset + DataLoader (tensors en formato [seq_len, batch])**

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

def numericalize_src(texto_en: str):
    """Convierte oración EN a ids con <bos> ... <eos>."""
    toks = tokenize_en(texto_en)
    ids = [SRC_VOCAB.bos_idx] + SRC_VOCAB.encode(toks) + [SRC_VOCAB.eos_idx]
    return torch.tensor(ids, dtype=torch.long)

def numericalize_trg(texto_de: str):
    """Convierte oración DE a ids con <bos> ... <eos>."""
    toks = tokenize_de(texto_de)
    ids = [TRG_VOCAB.bos_idx] + TRG_VOCAB.encode(toks) + [TRG_VOCAB.eos_idx]
    return torch.tensor(ids, dtype=torch.long)

class TranslationDataset(Dataset):
    def __init__(self, pares):
        self.pares = pares

    def __len__(self):
        return len(self.pares)

    def __getitem__(self, idx):
        src, trg = self.pares[idx]
        return numericalize_src(src), numericalize_trg(trg)

def collate_fn(batch):
    """Pad a la longitud máxima del batch. Salida: [seq_len, batch]."""
    src_list, trg_list = zip(*batch)
    src_pad = pad_sequence(src_list, padding_value=SRC_VOCAB.pad_idx)
    trg_pad = pad_sequence(trg_list, padding_value=TRG_VOCAB.pad_idx)
    return src_pad, trg_pad

train_ds = TranslationDataset(train_pairs)
valid_ds = TranslationDataset(valid_pairs)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_ds, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Sanity check de formas
src_b, trg_b = next(iter(train_loader))
print("SRC batch shape:", src_b.shape, " (src_len, batch)")
print("TRG batch shape:", trg_b.shape, " (trg_len, batch)")


#### **4. Entrenamiento Seq2Seq (SIN atención) + "overfit check"**

In [ ]:
import torch.nn as nn
import torch.optim as optim

def train_with_tf(modelo, iterator, optimizer, criterion, clip, teacher_forcing_ratio=1.0):
    """Entrena 1 época, pudiendo fijar teacher forcing ratio explícitamente."""
    modelo.train()
    epoch_loss = 0.0

    for src, trg in iterator:
        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()

        # Importante: usamos teacher_forcing_ratio controlado
        output = modelo(src, trg, teacher_forcing_ratio=teacher_forcing_ratio)

        output_dim = output.shape[-1]

        # Quitamos <bos> para loss
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].contiguous().view(-1)

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(modelo.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# Hiperparámetros
INPUT_DIM  = len(SRC_VOCAB)
OUTPUT_DIM = len(TRG_VOCAB)

ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
HID_DIM = 128
N_LAYERS = 1

# Dropout en 0 para que memorice fácil (solo para prueba)
ENC_DROPOUT = 0.0
DEC_DROPOUT = 0.0

CLIP = 1.0

# Modelo: usa TUS clases Encoder/Decoder/Seq2Seq
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT).to(device)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT).to(device)
model = Seq2Seq(enc, dec, device, TRG_VOCAB).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.NLLLoss(ignore_index=TRG_VOCAB.pad_idx)

# Overfit check: debería bajar fuerte la loss en train
for epoch in range(1, 201):
    train_loss = train_with_tf(model, train_loader, optimizer, criterion, CLIP, teacher_forcing_ratio=1.0)
    if epoch % 25 == 0:
        print(f"Epoca {epoch:03d} | train loss {train_loss:.3f}")


#### **5. Inferencia (greedy decoding) SIN atención**

In [ ]:
def translate_greedy(modelo, sentence_en: str, max_len=20):
    """Traduce EN->DE usando greedy decoding."""
    modelo.eval()
    with torch.no_grad():
        # src: [src_len] -> [src_len, 1]
        src = numericalize_src(sentence_en).unsqueeze(1).to(device)

        hidden, cell = modelo.encoder(src)

        # Comenzamos con <bos>
        input_tok = torch.tensor([TRG_VOCAB.bos_idx], device=device)

        out_ids = [TRG_VOCAB.bos_idx]

        for _ in range(max_len):
            output, hidden, cell = modelo.decoder(input_tok, hidden, cell)
            top1 = output.argmax(1).item()
            out_ids.append(top1)
            input_tok = torch.tensor([top1], device=device)

            if top1 == TRG_VOCAB.eos_idx:
                break

    tokens = TRG_VOCAB.decode(out_ids)[1:]  # quitamos <bos>
    if EOS in tokens:
        tokens = tokens[:tokens.index(EOS)]
    return tokens

# Probar sobre TRAIN (debería funcionar mejor tras overfit)
print("PRUEBAS (TRAIN)")
for s, _ in train_pairs[:6]:
    print(s, "->", " ".join(translate_greedy(model, s)))

print("\nPRUEBAS (VALID)")
for s, _ in valid_pairs:
    print(s, "->", " ".join(translate_greedy(model, s)))


#### **6. Evaluación BLEU (con smoothing para frases cortas)**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smooth = SmoothingFunction().method1

print("BLEU (VALID)")
for s, ref in valid_pairs:
    hyp = translate_greedy(model, s)
    bleu = sentence_bleu([tokenize_de(ref)], hyp, smoothing_function=smooth)
    print(f"EN: {s} | REF: {ref} | HYP: {' '.join(hyp)} | BLEU: {bleu:.3f}")


#### **7. Atención + Encoder/Decoder con atención**

In [ ]:
import matplotlib.pyplot as plt

class Attention(nn.Module):
    """Atención aditiva (tipo Bahdanau)."""
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: [n_layers, batch, hid_dim]
        # encoder_outputs: [src_len, batch, hid_dim]
        src_len = encoder_outputs.shape[0]

        # Tomamos la última capa del hidden
        hidden_last = hidden[-1].unsqueeze(0).repeat(src_len, 1, 1)  # [src_len, batch, hid_dim]

        energy = torch.tanh(self.attn(torch.cat((hidden_last, encoder_outputs), dim=2)))  # [src_len,batch,hid]
        scores = self.v(energy).squeeze(2).transpose(0, 1)  # [batch, src_len]
        return torch.softmax(scores, dim=1)  # [batch, src_len]


class EncoderAttn(nn.Module):
    """Encoder LSTM que devuelve outputs por token (para atención)."""
    def __init__(self, vocab_len, emb_dim, hid_dim, n_layers, dropout_prob):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_len, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, input_batch):
        embedded = self.dropout(self.embedding(input_batch)).to(device)  # [src_len,batch,emb]
        outputs, (hidden, cell) = self.lstm(embedded)                   # outputs: [src_len,batch,hid]
        return outputs, hidden, cell


class DecoderAttn(nn.Module):
    """Decoder LSTM que usa atención sobre encoder_outputs."""
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, attention: Attention):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        # El LSTM recibe [embedding + contexto]
        self.lstm = nn.LSTM(emb_dim + hid_dim, hid_dim, n_layers, dropout=dropout)

        # Proyección a vocab: usamos output + contexto + embedding
        self.fc_out = nn.Linear(hid_dim * 2 + emb_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, encoder_outputs):
        # input: [batch] -> [1,batch]
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))  # [1,batch,emb]

        # Pesos de atención: [batch, src_len]
        attn_weights = self.attention(hidden, encoder_outputs)          # [batch,src_len]
        attn_weights_bmm = attn_weights.unsqueeze(1)                    # [batch,1,src_len]

        # encoder_outputs a [batch,src_len,hid]
        enc = encoder_outputs.transpose(0, 1)
        context = torch.bmm(attn_weights_bmm, enc)                      # [batch,1,hid]
        context = context.transpose(0, 1)                               # [1,batch,hid]

        rnn_input = torch.cat((embedded, context), dim=2)               # [1,batch,emb+hid]
        output, (hidden, cell) = self.lstm(rnn_input, (hidden, cell))   # output: [1,batch,hid]

        output = output.squeeze(0)    # [batch,hid]
        context = context.squeeze(0)  # [batch,hid]
        embedded = embedded.squeeze(0)# [batch,emb]

        logits = self.fc_out(torch.cat((output, context, embedded), dim=1))
        prediction = self.softmax(logits)

        return prediction, hidden, cell, attn_weights  # attn_weights: [batch, src_len]


##### **7.1 Seq2Seq con atención**

In [ ]:
class Seq2SeqAttn(nn.Module):
    """Seq2Seq con atención: encoder devuelve outputs completos."""
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim
        assert encoder.n_layers == decoder.n_layers

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src)

        input = trg[0, :]  # <bos>

        for t in range(1, trg_len):
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs


##### **7.2 Entrenamiento con atención**

In [ ]:
attn = Attention(HID_DIM).to(device)

enc_a = EncoderAttn(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, 0.0).to(device)
dec_a = DecoderAttn(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, 0.0, attn).to(device)

model_attn = Seq2SeqAttn(enc_a, dec_a, device).to(device)

optimizer_a = optim.Adam(model_attn.parameters(), lr=1e-3)
criterion_a = nn.NLLLoss(ignore_index=TRG_VOCAB.pad_idx)

for epoch in range(1, 201):
    train_loss = train_with_tf(model_attn, train_loader, optimizer_a, criterion_a, CLIP, teacher_forcing_ratio=1.0)
    if epoch % 25 == 0:
        print(f"[Attn] Epoca {epoch:03d} | train loss {train_loss:.3f}")


#### **8. Inferencia con atención + heatmap de alineamientos**

In [ ]:
def translate_greedy_attn(modelo, sentence_en: str, max_len=20):
    """Traduce EN->DE y devuelve matriz de atención (para visualizar alineamientos)."""
    modelo.eval()
    attn_rows = []

    with torch.no_grad():
        src = numericalize_src(sentence_en).unsqueeze(1).to(device)  # [src_len,1]
        encoder_outputs, hidden, cell = modelo.encoder(src)

        input_tok = torch.tensor([TRG_VOCAB.bos_idx], device=device)
        out_ids = [TRG_VOCAB.bos_idx]

        for _ in range(max_len):
            output, hidden, cell, attn_w = modelo.decoder(input_tok, hidden, cell, encoder_outputs)
            top1 = output.argmax(1).item()
            out_ids.append(top1)

            # attn_w: [batch=1, src_len] -> guardamos [src_len]
            attn_rows.append(attn_w.squeeze(0).detach().cpu().numpy())

            input_tok = torch.tensor([top1], device=device)
            if top1 == TRG_VOCAB.eos_idx:
                break

    tokens = TRG_VOCAB.decode(out_ids)[1:]
    if EOS in tokens:
        tokens = tokens[:tokens.index(EOS)]

    attn_mat = np.stack(attn_rows, axis=0) if len(attn_rows) else None
    return tokens, attn_mat

#  Prueba + visualización
frase = "i like books"
hyp, attn_mat = translate_greedy_attn(model_attn, frase)
print(frase, "->", " ".join(hyp))

# Ejes del heatmap
src_tokens = [BOS] + tokenize_en(frase) + [EOS]
trg_tokens = hyp

if attn_mat is not None:
    plt.figure()
    plt.imshow(attn_mat, aspect="auto")
    plt.xticks(range(len(src_tokens)), src_tokens, rotation=45, ha="right")
    plt.yticks(range(len(trg_tokens)), trg_tokens)
    plt.xlabel("Tokens fuente (EN)  ->  (keys/values del encoder)")
    plt.ylabel("Pasos del decoder (DE)  ->  (queries)")
    plt.title("Alineamiento por atención (Bahdanau)")
    plt.tight_layout()
    plt.show()


#### **Conexión directa con la Atención y el Transformer**

Qué debes quedarte de Seq2Seq recurrente:

1. **Bottleneck**: comprimir toda la fuente en un único estado (o una trayectoria de estados) limita la capacidad.
2. **Long-range**: la señal de gradiente y la información útil "se diluyen" a medida que crece la longitud.
3. **Atención** resuelve ambos: en cada paso, el decodificador **consulta** (pondera) todos los estados del codificador.


### **Ejercicios**

Aquí tienes **enunciados de ejercicios** que puedes insertar directamente en el cuaderno, alineados con **cada tema** (Perceptrón/MLP, activaciones/pérdida, backprop/gradientes, regularización, optimizadores+scheduler, RNN/LSTM y motivación a atención).

#### **1. Perceptrón, MLP, activaciones y función de pérdida**

1. **Perceptrón -> Regresión logística (clasificación binaria).** Modifica el perceptrón "a mano" para que use salida sigmoide y entrene minimizando *binary cross-entropy*. Reporta: curva de pérdida, accuracy y frontera de decisión.
2. **MLP en `make_moons`: capacidad vs sobreajuste.** Entrena el MLP variando el número de neuronas ocultas (por ejemplo, 8, 32, 128) y la profundidad (1 vs 3 capas). Entrega una tabla con *train acc*, *val acc* y brecha de generalización.
3. **Función de pérdida y clases desbalanceadas.** Crea un desbalance artificial en `make_moons` (submuestreo de una clase) y compara: (a) CrossEntropy estándar, (b) pérdida ponderada por clase. Reporta métricas por clase (precision/recall/F1).

#### **2. Entrenamiento y retropropagación: papel de los gradientes**

4. **Ablación de gradientes: ¿qué capas aprenden primero?** Durante el entrenamiento del MLP, registra por época la norma del gradiente de cada capa (pesos) y grafica su evolución. Interpreta qué capas dominan el aprendizaje y por qué.
5. **Diagnóstico de saturación.** Repite el entrenamiento sustituyendo ReLU por sigmoid/tanh y mide: (a) norma promedio del gradiente por época, (b) tiempo hasta alcanzar una accuracy objetivo. Explica la relación entre activación y vanishing gradients.
6. **Escala de inicialización.** Cambia la inicialización de pesos (más pequeña vs más grande) y evalúa estabilidad del entrenamiento (exploding/vanishing). Reporta un gráfico de pérdida y norma de gradiente.

#### **3. Verificación de retropropagación (diferencias finitas)**

7. **Gradient check robusto.** Extiende la verificación por diferencias finitas para probar **N parámetros aleatorios** (por ejemplo, 50) en distintas capas y reporta el **máximo error relativo**. Compara resultados en `float32` vs `float64`.
8. **Detectar un bug de autograd.** Introduce intencionalmente un error (por ejemplo,, `detach()` en una activación o en una rama del cómputo) y muestra que el gradient check falla. Explica qué cambió y por qué.

#### **4. Regularización y generalización (L2, dropout, batch norm, early stopping)**

9. **Comparación sistemática (tabla obligatoria).** Entrena 4 variantes del MLP: (a) baseline, (b) L2/weight decay, (c) dropout, (d) batch norm. Mantén el resto fijo. Entrega una tabla con: mejor *val acc*, época del mejor modelo, brecha train–val, y comentario breve.
10. **Early stopping con sensibilidad.** Implementa early stopping con `patience ∈ {3, 10, 20}` y `min_delta` (por ejemplo, 0 y 1e-3). Reporta cómo cambia: (a) época de parada, (b) val acc final, (c) varianza entre corridas.
11. **Regularización y calibración.** Evalúa calibración del clasificador (por ejemplo,, ECE o reliability diagram) para baseline vs L2 vs dropout. Concluye cuál regulariza mejor "probabilidades" y no solo accuracy.

#### **5. Optimizadores modernos y programación de tasa de aprendizaje**

12. **SGD+momentum vs Adam vs AdamW (fair comparison).** Para cada optimizador, realiza una pequeña búsqueda de LR (por ejemplo, 3 valores) y reporta: mejor configuración, curvas de pérdida y val acc. Concluye cuál converge más rápido y cuál generaliza mejor.
13. **Scheduler: cosine con y sin warmup.** Añade warmup (por ejemplo, 5% de épocas) antes del cosine annealing. Compara: estabilidad inicial, pérdida temprana y rendimiento final.
14. **Weight decay: Adam vs AdamW.** Manteniendo el mismo `weight_decay`, compara Adam vs AdamW y discute por qué no son equivalentes. Reporta diferencias en norma de pesos y brecha train-val.

#### **6. Limitaciones de RNN/LSTM y motivación hacia atención**

15. **Norma de gradiente vs longitud: RNN vs LSTM vs GRU.** Repite el experimento de secuencias largas con los tres modelos y grafica norma de gradiente vs longitud. Concluye cuál es más estable y en qué rango de longitudes.
16. **Seq2Seq sin atención: "overfit check" + generalización.** (a) Fuerza overfit en un subconjunto mínimo y verifica que BLEU se acerque a 1.0 en train. (b) Luego sube el tamaño del dataset y mide BLEU en validación; analiza errores típicos (orden, palabras desconocidas, omisiones).
17. **Ablación de atención: impacto real.** Entrena el modelo Seq2Seq **con** y **sin** atención bajo el mismo presupuesto (épocas/params). Reporta BLEU y 3 ejemplos cualitativos donde atención mejora (y 1 donde no).
18. **Heatmaps de alineamiento: diagnóstico.** Genera heatmaps para al menos 5 frases y etiqueta patrones: alineamiento diagonal, saltos, repeticiones. Propón un cambio (por ejemplo, dropout en atención o ajuste de capacidad) y verifica si mejora los patrones.


In [ ]:
## Tus respuestas